In [1]:
# Pandas is used for data manipulation
import pandas as pd
# Read in data and display first 5 rows
dogs = pd.read_csv('D:/projects/20151001hundehalter.csv')
dogs.head(5)

,HALTER_ID,ALTER,GESCHLECHT,STADTKREIS,STADTQUARTIER,RASSE1,RASSE1_MISCHLING,RASSE2,RASSE2_MISCHLING,RASSENTYP,GEBURTSJAHR_HUND,GESCHLECHT_HUND,HUNDEFARBE
0,126,51-60,m,9.0,92.0,Welsh Terrier,NaN,NaN,NaN,K,2011,w,schwarz/braun
1,574,61-70,w,2.0,23.0,Cairn Terrier,NaN,NaN,NaN,K,2002,w,brindle
2,695,41-50,m,6.0,63.0,Labrador Retriever,NaN,NaN,NaN,I,2012,w,braun
3,893,61-70,w,7.0,71.0,Mittelschnauzer,NaN,NaN,NaN,I,2010,w,schwarz
4,1177,51-60,m,10.0,102.0,Shih Tzu,NaN,NaN,NaN,K,2011,m,schwarz/weiss


In [2]:
print('The shape of our features is:', dogs.shape)

The shape of our features is: (6980, 13)


In [3]:
dogs = dogs[["HALTER_ID", 'ALTER', 'GESCHLECHT', 'STADTKREIS', 'STADTQUARTIER','RASSE1']]

In [4]:
dogs = dogs.rename(columns={'HALTER_ID': 'HOLDER_ID',
                            'ALTER': 'AGE',
                            'GESCHLECHT': 'GENDER',
                            'STADTKREIS': 'CITY DISTRICT',
                            'STADTQUARTIER': 'CITY QUARTER',
                           'RASSE1':'BREED'})
dogs.head(5)

,HOLDER_ID,AGE,GENDER,CITY DISTRICT,CITY QUARTER,BREED
0,126,51-60,m,9.0,92.0,Welsh Terrier
1,574,61-70,w,2.0,23.0,Cairn Terrier
2,695,41-50,m,6.0,63.0,Labrador Retriever
3,893,61-70,w,7.0,71.0,Mittelschnauzer
4,1177,51-60,m,10.0,102.0,Shih Tzu


In [5]:
dogs.corr()

,HOLDER_ID,CITY DISTRICT,CITY QUARTER
HOLDER_ID,1.00000,0.018380,0.021120
CITY DISTRICT,0.01838,1.000000,0.997864
CITY QUARTER,0.02112,0.997864,1.000000


In [14]:
my_series = dogs.duplicated()
my_series

0       False
1       False
2       False
3       False
4       False
        ...  
6975    False
6976    False
6977    False
6978    False
6979    False
Length: 6978, dtype: bool

In [13]:
num_dupes = 0
my_series = dogs.duplicated()
for x in dogs.index:
    if my_series[x]:
        num_dupes+= 1
        
print("There are", num_dupes, "duplicate entries")

There are 358 duplicate entries


In [8]:
#unduped_dogs = dogs.drop_duplicates(subset=['HOLDER_ID'])
#unduped_dogs

unduped_dogs = dogs.drop_duplicates()

,HOLDER_ID,AGE,GENDER,CITY DISTRICT,CITY QUARTER,BREED
0,126,51-60,m,9.0,92.0,Welsh Terrier
1,574,61-70,w,2.0,23.0,Cairn Terrier
2,695,41-50,m,6.0,63.0,Labrador Retriever
3,893,61-70,w,7.0,71.0,Mittelschnauzer
4,1177,51-60,m,10.0,102.0,Shih Tzu
...,...,...,...,...,...,...
6974,128462,21-30,w,5.0,52.0,Old Englische Bulldogge
6975,128471,21-30,w,8.0,83.0,Tibetan Spaniel
6977,128479,51-60,w,6.0,63.0,Lagotto Romagnolo
6978,128482,21-30,w,9.0,91.0,Yorkshire Terrier


In [17]:
#Create function to convert a string of form 'x-y' into a float = average(x,y)
import statistics 

def average_age(txt):
    ages = [int(s) for s in txt.split('-') if s.isdigit()]
    return statistics.mean(ages)

print(average_age("12-18-12-a-234-a-b"))

69


In [10]:
#Make sure AGE values have values and are not empty
dogs = dogs[dogs['AGE'].notnull()]
dogs

,HOLDER_ID,AGE,GENDER,CITY DISTRICT,CITY QUARTER,BREED
0,126,51-60,m,9.0,92.0,Welsh Terrier
1,574,61-70,w,2.0,23.0,Cairn Terrier
2,695,41-50,m,6.0,63.0,Labrador Retriever
3,893,61-70,w,7.0,71.0,Mittelschnauzer
4,1177,51-60,m,10.0,102.0,Shih Tzu
...,...,...,...,...,...,...
6975,128471,21-30,w,8.0,83.0,Tibetan Spaniel
6976,128471,21-30,w,8.0,83.0,Rottweiler
6977,128479,51-60,w,6.0,63.0,Lagotto Romagnolo
6978,128482,21-30,w,9.0,91.0,Yorkshire Terrier


In [11]:
averaged_dogs = dogs['AGE'].apply(lambda x: average_age(x))
dogs['AGE'] = averaged_dogs
dogs

,HOLDER_ID,AGE,GENDER,CITY DISTRICT,CITY QUARTER,BREED
0,126,55.5,m,9.0,92.0,Welsh Terrier
1,574,65.5,w,2.0,23.0,Cairn Terrier
2,695,45.5,m,6.0,63.0,Labrador Retriever
3,893,65.5,w,7.0,71.0,Mittelschnauzer
4,1177,55.5,m,10.0,102.0,Shih Tzu
...,...,...,...,...,...,...
6975,128471,25.5,w,8.0,83.0,Tibetan Spaniel
6976,128471,25.5,w,8.0,83.0,Rottweiler
6977,128479,55.5,w,6.0,63.0,Lagotto Romagnolo
6978,128482,25.5,w,9.0,91.0,Yorkshire Terrier
